In [15]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.integrate import cumulative_trapezoid as cumtrapz

In [16]:
df = pd.read_csv("Natixis.csv", sep = ";")

df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
df.sort_values("date", inplace = True)

df["value"] = (df["value"].astype(str).str.replace(",", ".", regex=False))
df["value"] = pd.to_numeric(df["value"], errors="coerce")

df["return"] = df["value"] / df["value"].shift(1) - 1
df

,date,value,return
0,2015-01-02,5.621,NaN
1,2015-01-05,5.424,-0.035047
2,2015-01-06,5.329,-0.017515
3,2015-01-07,5.224,-0.019704
4,2015-01-08,5.453,0.043836
...,...,...,...
1018,2018-12-21,4.045,-0.001481
1019,2018-12-24,4.010,-0.008653
1020,2018-12-27,3.938,-0.017955
1021,2018-12-28,4.088,0.038090


## Question A (Ex2, part of Q1 and of Q2 of TD1)

**a** – From the time series of the daily prices of the stock Natixis between January 2015 and December 2016, provided with TD1, estimate a historical VaR on price returns at a one-day horizon for a given probability level (this probability is a parameter which must be changed easily). You must base your VaR on a non-parametric distribution (biweight Kernel, that is $K$ is the derivative of the logistic function $x \mapsto \frac{15}{16}(1-x^2)^2 \mathbb{1}_{|x| \leq 1}$).

In [17]:
df_2015_2016 = df[df["date"] < "2017"].dropna().loc[:, "return"]
df_2015_2016

1     -0.035047
2     -0.017515
3     -0.019704
4      0.043836
5     -0.020723
         ...   
508   -0.008118
509    0.000744
510   -0.000186
511   -0.009481
512    0.006006
Name: return, Length: 512, dtype: float64

Ce qu'on a fait (tout à refaire)

In [18]:
alpha = 0.99

def Kde_VaR(df, alpha, bw=None):
    # Fit KDE (Gaussian kernel)
    kde = gaussian_kde(df, bw_method=bw)  # 'scott' by default

    # Build a grid covering the tail well
    mu, s = np.mean(df), np.std(df, ddof=1)
    lo = min(df.min(), mu - 6*s)
    hi = max(df.max(), mu + 6*s)
    grid = np.linspace(lo, hi, 20001)

    # PDF on grid and CDF by numerical integration (trapezoid)
    pdf = kde(grid)
    cdf = cumtrapz(pdf, grid, initial=0.0)
    cdf = cdf / cdf[-1]  # normalize to 1

    # α-quantile by interpolation of the CDF
    q_alpha = np.interp(alpha, cdf, grid)

    return -q_alpha

kde_var = Kde_VaR(df_2015_2016, alpha)

print("The empirical VaR is for returns between 2015 and 2016 is : ", kde_var*100, "%")
print("There are", round(kde_var*100, 2), "% chances to lose", round((1 - alpha)*100, 2), "% of the portfolio's value")

The empirical VaR is for returns between 2015 and 2016 is :  -6.007630146624207 %
There are -6.01 % chances to lose 1.0 % of the portfolio's value


### First step: Estimation of the kernel density 
$$\hat{f}(x) = \frac{1}{nh}\sum_{i=1}^{n} K\left(\frac{x - X_i}{h}\right)$$ with $$K(x) = \frac{15}{16}(1-x^2)^2 \mathbb{1}_{|x| \leq 1}$$

In [ ]:
#def K(x)

#def f_hat(x, h, tab_returns)



### Second Step : Choice of h

- Tracer Graph de la densité pour different h (voir comment la densité évolue)
- Utiliser un résultat théorique pour le h optimal (papier de recherche etc)

### 3rd Step : Define the CDF of K and use

$$\hat{F}(x) = \frac{1}{n}\sum_{i=1}^{n} \mathcal{K}\left(\frac{x - X_i}{h}\right)$$

To define $\mathcal{K}$ we have to primitive K.

$$
\mathcal{K}(u) = \begin{cases}
0 & \text{si } u < -1 \\[0.5em]
\frac{1}{2} + \frac{15}{16}\left(u - \frac{2u^3}{3} + \frac{u^5}{5}\right) & \text{si } -1 \leq u \leq 1 \\[0.5em]
1 & \text{si } u > 1
\end{cases}
$$

In [ ]:
#def K_cdf(x):

#def F_hat(x, h, tab_returns):

### Step 4: 



In [19]:
def kernel_VaR(X, h, alpha, number_of_points):
    # 1. Calculer F_hat(x) pour tous les points x
    x, y = cumulative_kernel_density(X, h, number_of_points)
    # x = grille de valeurs (1000 points entre min et max des rendements)
    # y = F_hat(x) pour chaque point (valeurs de la CDF)
    
    # 2. Trouver l'index où y >= alpha pour la première fois
    y_VaR = np.argmax(y >= alpha)
    # argmax retourne l'INDEX du premier True dans le tableau booléen
    # Exemple: si alpha=0.05, on cherche le premier point où F_hat >= 0.05
    
    # 3. Retourner la valeur de x correspondant à cet index
    return x[y_VaR]

**b – Which proportion of price returns between January 2017 and December 2018 does exceed the VaR
threshold defined in the previous question? Do you validate the choice of this non-parametric VaR?**


In [20]:
df_2017_2018 = df[df["date"] >= "2017"].dropna().loc[:, "return"]
df_2017_2018

513     0.007463
514     0.040741
515     0.003737
516    -0.008155
517    -0.005719
          ...   
1018   -0.001481
1019   -0.008653
1020   -0.017955
1021    0.038090
1022    0.007583
Name: return, Length: 510, dtype: float64

Simply find the proportion of returns < to the VaR find and compare with the alpha. Mentionne the word coverage.

In [ ]:
#j'ai simplifié la fonction qu'on avait dans notr elab car une seul VaR
def Proportion(df, VaR):
    prop = df[df < VaR].count() / df.count()
    return prop

**Question B (Ex2, Q5 of TD2)
Calculate the expected shortfall for the VaR calculated in question A. How is the result, compared to
the VaR?**

In [ ]:
losses_beyond_VaR = df_2015_2016[df_2015_2016["returns"] <= VaR_alpha]
ES_alpha = np.mean(losses_beyond_VaR)

**Question C (Ex2, Q1 and Q2 of TD3)

With the dataset provided for TD1 on Natixis prices, first calculate daily returns. You will then analyse
these returns using a specific method in the field of the EVT.

a – Estimate the GEV parameters for the two tails of the distribution of returns, using the estimator of
Pickands. What can you conclude about the nature of the extreme gains and losses?**


In [23]:
# Ce qu'on a fait TD3

# loss
loss = list(df[df["return"] < 0]["return"] * -1)
loss.sort()
n = len(loss)
e_loss = np.log((loss[int(n-1 - np.floor(np.log(n)) + 1)] - loss[int(n-1 - 2 * np.floor(np.log(n)) + 1)]) / (loss[int(n-1 - 2 * np.floor(np.log(n)) + 1)] - loss[int(n-1 - 4 * np.floor(np.log(n)) + 1)]))
e_loss /= np.log(2)
print("e_loss :", e_loss)

# gain
gain = list(df[df["return"] > 0]["return"] * -1)
gain.sort()
n = len(gain)
e_gain = np.log((gain[int(n-1 - np.floor(np.log(n)) + 1)] - gain[int(n-1 - 2 * np.floor(np.log(n)) + 1)]) / (gain[int(n-1 - 2 * np.floor(np.log(n)) + 1)] - gain[int(n-1 - 4 * np.floor(np.log(n)) + 1)]))
e_gain /= np.log(2)
print("e_gain :", e_gain)

e_loss : -0.5089715779341932
e_gain : -0.9367427921518837


**b – Calculate the value at risk based on EVT for various confidence levels, with the assumption of iid
returns.**
